## Sentinel-2 crop

In [1]:
wkt = 'POLYGON((-6.79440295589744 37.1191319871777,-6.80827154725944 37.1673031942056,-6.72121114356795 37.2425767108835,-6.71042775152577 37.2930498879682,-6.73405462341917 37.3301447063348,-6.71685147187095 37.3522253716156,-6.67435624435345 37.3534492963332,-6.64461873043217 37.3365633914571,-6.62625336367183 37.3569789562915,-6.59581018720718 37.3506994424118,-6.58125246058411 37.3703051383248,-6.492413660497 37.3590379521576,-6.4716624422663 37.3807359777278,-6.43859705183134 37.368908852933,-6.44398095303713 37.347088527492,-6.42200724935979 37.3329929700694,-6.37035012694059 37.3583637388823,-6.33153540861988 37.3329986280188,-6.25703879672152 37.3486795386779,-6.2123425985781 37.3266608721382,-6.1976208676784 37.283024780795,-6.17366048732846 37.2660742144794,-6.15177710683581 37.2667431439207,-6.14475916204647 37.3055642119627,-6.1153577846492 37.3105672777454,-6.04475036124726 37.2771976083931,-6.04063041298302 37.2060457158668,-6.01561413928283 37.1744528799079,-6.03401959555254 37.1582677514823,-6.01528760169561 37.1211642619874,-6.03587498721126 37.0508366627984,-6.08415258431292 37.0557470251154,-6.1443715598251 37.0182321851901,-6.20489081698833 36.9013662293736,-6.24848183495607 36.8846441975983,-6.28133674315331 36.9000390906331,-6.27285487885334 36.8641095189495,-6.21240564569216 36.8569441225281,-6.19273049982008 36.8273222937041,-6.23650318117893 36.7815132843624,-6.25972339425557 36.7794168824025,-6.29452828553456 36.7134050019461,-6.32211152509425 36.693679078589,-6.40354402587554 36.7535989999008,-6.39366914786855 36.7750231637474,-6.42465019950879 36.7928589511795,-6.48913510482893 36.9063367348853,-6.55574132052792 36.9640015939793,-6.54729285159512 36.9812830814621,-6.79440295589744 37.1191319871777))'

In [2]:
import os

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import snappy
%matplotlib inline
import dateutil.parser as parser
import gc
from datetime import datetime

In [3]:
data_path = "/workspace/data"

In [4]:
s2_identifier = 'S2A_MSIL1C_20170909T110651_N0205_R137_T30STF_20170909T111217'

In [5]:
s2prd = "%s/%s/%s.SAFE/MTD_MSIL1C.xml" % (data_path, s2_identifier, s2_identifier)
product = snappy.ProductIO.readProduct(s2prd)

width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()
name = product.getName()
description = product.getDescription()
band_names = product.getBandNames()

In [6]:
bands_10 = 'B2,B3,B4,B8'
bands_20 = 'B5,B6,B7,B8A,B11,B12'
bands_60 = 'B1,B9,B10'

In [9]:
snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

HashMap = snappy.jpy.get_type('java.util.HashMap')

parameters = HashMap()
parameters.put('referenceBand', 'B2')

resample_10 = snappy.GPF.createProduct('Resample', parameters, product)

HashMap = snappy.jpy.get_type('java.util.HashMap')    

parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('sourceBands', bands_10)

subset_radio_10 = snappy.GPF.createProduct('Subset', parameters, resample_10)

In [10]:
snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

HashMap = snappy.jpy.get_type('java.util.HashMap')

parameters = HashMap()
parameters.put('referenceBand', 'B5')

resample_20 = snappy.GPF.createProduct('Resample', parameters, product)

HashMap = snappy.jpy.get_type('java.util.HashMap')    

parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('sourceBands', bands_20)

subset_radio_20 = snappy.GPF.createProduct('Subset', parameters, resample_20)

In [11]:
snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

HashMap = snappy.jpy.get_type('java.util.HashMap')

parameters = HashMap()
parameters.put('referenceBand', 'B1')

resample_60 = snappy.GPF.createProduct('Resample', parameters, product)

HashMap = snappy.jpy.get_type('java.util.HashMap')    

parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('sourceBands', bands_60)

subset_radio_60 = snappy.GPF.createProduct('Subset', parameters, resample_60)

In [12]:
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')

geom = WKTReader().read(wkt)

HashMap = snappy.jpy.get_type('java.util.HashMap')    

parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('geoRegion', geom)

subset_geo_10 = snappy.GPF.createProduct('Subset', parameters, subset_radio_10)

In [13]:
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')

geom = WKTReader().read(wkt)

HashMap = snappy.jpy.get_type('java.util.HashMap')    

parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('geoRegion', geom)

subset_geo_20 = snappy.GPF.createProduct('Subset', parameters, subset_radio_20)

In [14]:
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')

geom = WKTReader().read(wkt)

HashMap = snappy.jpy.get_type('java.util.HashMap')    

parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('geoRegion', geom)

subset_geo_60 = snappy.GPF.createProduct('Subset', parameters, subset_radio_60)

In [16]:
snappy.ProductIO.writeProduct(subset_geo_10, s2_identifier + '_10m.tif', 'GeoTIFF')
snappy.ProductIO.writeProduct(subset_geo_20, s2_identifier + '_20m.tif', 'GeoTIFF')
snappy.ProductIO.writeProduct(subset_geo_60, s2_identifier + '_60m.tif', 'GeoTIFF')